<a href="https://colab.research.google.com/github/batarg1nroman/Batargin-Roman/blob/main/first_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime, timedelta
from abc import ABCMeta, ABC, abstractmethod
from functools import wraps
import json
from typing import Dict, Any, List
import logging

class InvalidDateError(Exception):
    """Исключение, возникающее при некорректных датах проекта (дата окончания раньше даты начала)."""
    """Args:start_date: Дата начала проекта. end_date: Дата окончания проекта."""
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        message = f"Дата окончания ({end_date}) не может быть раньше даты начала ({start_date})"
        super().__init__(message)


def validate_project_dates(start_date, end_date):
    """Проверяет, что дата окончания не раньше даты начала.

    Args:
        start_date: Дата начала проекта.
        end_date: Дата окончания проекта.

    Raises:
        InvalidDateError: Если end_date < start_date.
    """
    if end_date < start_date:
        raise InvalidDateError(start_date, end_date)


class PermissionDeniedError(Exception):
    """Исключение, возникающее при отсутствии прав у пользователя."""
    def __init__(self, message="Недостаточно прав для выполнения операции"):
        self.message = message
        """
        Args:
            message: Сообщение об ошибке.
        """
        super().__init__(self.message)


def check_permissions(required_permission: str):
    """Декоратор для проверки прав пользователя перед выполнением функции.

    Args:
        required_permission: Требуемое право (например, "edit_project").

    Returns:
        Декорированная функция.

    Raises:
        PermissionDeniedError: Если у пользователя нет нужного права.
    """
    def decorator(func):
        @wraps(func)
        def wrapper(user, *args, **kwargs):
            if not has_permission(user, required_permission):
                raise PermissionDeniedError(
                    f"У пользователя {user['name']} нет права {required_permission}"
                )
            return func(user, *args, **kwargs)
        return wrapper
    return decorator


def has_permission(user: dict, permission: str):
    return permission in user.get('permissions', [])


class BudgetHandler(ABC):
    """Абстрактный класс для обработки запросов на изменение бюджета."""
    """Устанавливает следующий обработчик в цепочке.

        Args:
            decisionMaker: Следующий обработчик (BudgetHandler).

        Returns:
            Установленный обработчик.
    """
    def __init__(self):
        self._next_decisionMaker = None

    def set_next(self, decisionMaker):
        self._next_decisionMaker = decisionMaker
        return decisionMaker

    @abstractmethod
    def decision_request(self, request):
        """Обрабатывает запрос на изменение бюджета.

        Args:
            request: Словарь с данными запроса (должен содержать "amount").
        """
        pass


"""
Модуль для управления проектами с системой цепочки ответственности за бюджет,
фабрикой проектов, отслеживанием прогресса и уведомлениями.
"""

class ProjectLeader(BudgetHandler):
    """Обработчик запросов на изменение бюджета для сумм до 1000 единиц."""

    def decision_request(self, request: dict):
        """Обрабатывает запрос на изменение бюджета.

        Args:
            request: Словарь с данными запроса, должен содержать ключ 'amount'.

        Behavior:
            - Одобряет запросы до 1000 единиц
            - Передает запросы выше 1000 следующему обработчику
            - Если следующего обработчика нет - отклоняет запрос
        """
        if request["amount"] <= 1000:
            print(f"✅ Руководитель проекта одобрил изменение бюджета на {request['amount']} единиц")
        elif self._next_decisionMaker:
            self._next_decisionMaker.decision_request(request)
        else:
            print("❌ Запрос не может быть обработан")


class FinanceDepartment(BudgetHandler):
    """Обработчик запросов на изменение бюджета для сумм от 1000 до 5000 единиц."""

    def decision_request(self, request: dict):
        """Обрабатывает запрос на изменение бюджета.

        Args:
            request: Словарь с данными запроса, должен содержать ключ 'amount'.

        Behavior:
            - Одобряет запросы от 1000 до 5000 единиц
            - Передает запросы выше 5000 следующему обработчику
            - Если следующего обработчика нет - отклоняет запрос
        """
        if 1000 < request["amount"] <= 5000:
            print(f"✅ Финансовый отдел одобрил изменение бюджета на {request['amount']} единиц")
        elif self._next_decisionMaker:
            self._next_decisionMaker.decision_request(request)
        else:
            print("❌ Запрос не может быть обработан")


class Director(BudgetHandler):
    """Обработчик запросов на изменение бюджета для сумм свыше 5000 единиц."""

    def decision_request(self, request: dict):
        """Обрабатывает запрос на изменение бюджета.

        Args:
            request: Словарь с данными запроса, должен содержать ключ 'amount'.

        Behavior:
            - Одобряет запросы до 10000 единиц
            - Отклоняет все остальные запросы как неожиданные
        """
        if request["amount"] < 10000:
            print(f"✅ Директор одобрил изменение бюджета на {request['amount']} единиц")
        else:
            print("❌ Неожиданный запрос для директора")


class ProjectFactory:
    """Фабрика для создания проектов различных типов."""

    @staticmethod
    def create_project(project_type: str, *args, **kwargs):
        """Создает проект указанного типа.

        Args:
            project_type: Тип проекта ('software', 'marketing', 'research')
            *args: Позиционные аргументы для конструктора проекта
            **kwargs: Именованные аргументы для конструктора проекта

        Returns:
            Созданный экземпляр проекта

        Raises:
            ValueError: Если передан неизвестный тип проекта
        """
        project_class = {
            "software": SoftwareProject,
            "marketing": MarketingProject,
            "research": ResearchProject,
        }.get(project_type.lower())

        if not project_class:
            raise ValueError(f"Неизвестный тип проекта: {project_type}")

        return project_class(*args, **kwargs)


class ProjectMeta(ABCMeta):
    """Метакласс для регистрации типов проектов."""

    _registry = {}

    def __new__(cls, name: str, bases: tuple, namespace: dict):
        """Создает новый класс и регистрирует его, если он наследуется от Project.

        Args:
            name: Имя создаваемого класса
            bases: Кортеж базовых классов
            namespace: Словарь пространства имен класса

        Returns:
            Новый класс
        """
        new_class = super().__new__(cls, name, bases, namespace)

        if bases and bases[0].__name__ == "Project":
            cls._registry[name] = new_class

        return new_class

    @classmethod
    def get_project_class(cls, name: str):
        """Возвращает класс проекта по его имени.

        Args:
            name: Имя класса проекта

        Returns:
            Класс проекта или None, если не найден
        """
        return cls._registry.get(name)

    @classmethod
    def create_project(cls, project_type: str, *args, **kwargs):
        """Создает проект указанного типа.

        Args:
            project_type: Тип создаваемого проекта
            *args: Позиционные аргументы
            **kwargs: Именованные аргументы

        Returns:
            Созданный экземпляр проекта

        Raises:
            ValueError: Если тип проекта неизвестен
        """
        project_class = cls.get_project_class(project_type)
        if project_class:
            return project_class(*args, **kwargs)
        raise ValueError(f"Неизвестный тип проекта: {project_type}")


class Trackable(ABC):
    """Интерфейс для отслеживания прогресса проекта."""

    @abstractmethod
    def track_progress(self):
        """Возвращает строку с информацией о прогрессе проекта.

        Returns:
            Строка с описанием текущего прогресса
        """
        pass


class Reportable(ABC):
    """Интерфейс для генерации отчетов."""

    @abstractmethod
    def generate_report(self) -> dict:
        """Генерирует отчет по проекту.

        Returns:
            Словарь с данными отчета
        """
        pass


class LoggingMixin:
    """Миксин для логирования действий проекта."""

    def log_action(self, action: str):
        """Логирует действие по проекту.

        Args:
            action: Описание выполненного действия
        """
        print(f"🔔 [Лог] Проект '{self.name}' обновлен: {action}")


class NotificationMixin:
    """Миксин для отправки уведомлений по проекту."""

    def send_notification(self, message: str) -> None:
        """Отправляет уведомление о событии проекта.

        Args:
            message: Текст уведомления
        """
        print(f"🔔 [Уведомление] Проект '{self.name}': {message}")


class Project(metaclass=ProjectMeta):
    """Базовый класс проекта."""

    def __init__(self, name: str, startDate: datetime, endDate: datetime,
                 budget: float, status: str, team: 'Team'):
        """Инициализирует проект.

        Args:
            name: Название проекта
            startDate: Дата начала проекта
            endDate: Дата окончания проекта
            budget: Бюджет проекта
            status: Текущий статус проекта
            team: Команда проекта
        """
        self.name = name
        self.startDate = startDate
        self.endDate = endDate
        self.budget = budget
        self.status = status
        self.team = team

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует проект в словарь."""
        return {
            'type': 'Project',
            'name': self.name,
            'start_date': self.startDate.isoformat(),
            'end_date': self.endDate.isoformat(),
            'budget': self.budget,
            'status': self.status,
            'team': self.team.to_dict()  # Используем метод to_dict для Team
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Project':
        """Создает проект из словаря."""
        team = Team.from_dict(data['team'])  # Используем метод from_dict для Team
        return cls(
            name=data['name'],
            startDate=datetime.fromisoformat(data['start_date']),
            endDate=datetime.fromisoformat(data['end_date']),
            budget=data['budget'],
            status=data['status'],
            team=team
        )

    def days_between_dates(self, date1: datetime, date2: datetime) -> timedelta:
        """Вычисляет разницу между двумя датами.

        Args:
            date1: Первая дата
            date2: Вторая дата

        Returns:
            Разница между датами как timedelta
        """
        delta = date2 - date1
        return abs(delta)

    def calculate_progress(self):
        """Вычисляет процент выполнения проекта.

        Returns:
            Процент выполнения (0-100)
        """
        fixed = self.startDate + (self.endDate - self.startDate) / 2
        total_days = self.days_between_dates(self.startDate, self.endDate).days
        elapsed_days = self.days_between_dates(self.startDate, fixed).days
        return min(100, (elapsed_days / total_days) * 100) if total_days > 0 else 0

    def track_progress(self):
        """Возвращает строку с информацией о прогрессе.

        Returns:
            Строка с процентом выполнения
        """
        return f"Прогресс: {self.calculate_progress():.1f}% завершено"

    def __str__(self):
        """Строковое представление проекта.

        Returns:
            Строка с названием и статусом проекта
        """
        return f"Название проекта: {self.name}. Статус: {self.status}."

    def __lt__(self, other: 'Project'):
        """Сравнение проектов по бюджету.

        Args:
            other: Другой проект для сравнения

        Returns:
            True если текущий проект имеет меньший бюджет
        """
        return self.budget < other.budget

    def __gt__(self, other: 'Project'):
        """Сравнение проектов по длительности.

        Args:
            other: Другой проект для сравнения

        Returns:
            True если текущий проект короче по времени
        """
        return other.days_between_dates(other.startDate, other.endDate) > \
               self.days_between_dates(self.startDate, self.endDate)

    def __eq__(self, other: object):
        """Сравнение проектов по бюджету и длительности.

        Args:
            other: Объект для сравнения

        Returns:
            True если проекты равны по бюджету и длительности
        """
        if not isinstance(other, Project):
            return NotImplemented
        return (
            self.budget == other.budget
            and self.days_between_dates(self.startDate, self.endDate) ==
               other.days_between_dates(other.startDate, other.endDate)
        )


class SoftwareProject(Project, LoggingMixin, NotificationMixin):
    """Класс программного проекта."""

    def __init__(self, name: str, startDate: datetime, endDate: datetime,
                 budget: float, status: str, team: 'Team', programming_languages: List[str]):
        """Инициализирует программный проект.

        Args:
            programming_languages: Список используемых языков программирования
        """
        super().__init__(name, startDate, endDate, budget, status, team)
        self.programming_languages = programming_languages

    def to_dict(self) -> Dict[str, Any]:
        data = super().to_dict()
        data.update({
            'type': 'SoftwareProject',
            'programming_languages': self.programming_languages
        })
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'SoftwareProject':
        team = Team.from_dict(data['team'])  # Десериализация Team
        return cls(
            name=data['name'],
            startDate=datetime.fromisoformat(data['start_date']),
            endDate=datetime.fromisoformat(data['end_date']),
            budget=data['budget'],
            status=data['status'],
            team=team,
            programming_languages=data['programming_languages']
        )

    def calculate_progress(self) -> float:
        """Вычисляет прогресс с учетом сложности языков программирования.

        Returns:
            Процент выполнения (0-100)
        """
        base_progress = super().calculate_progress()
        lang_factor = 1 - (len(self.programming_languages) * 0.05)
        return min(100, base_progress * lang_factor)

    def update_languages(self, new_languages: List[str]) -> None:
        """Обновляет список языков программирования.

        Args:
            new_languages: Новый список языков
        """
        old_languages = self.programming_languages
        self.programming_languages = new_languages
        self.log_action(f"Языки изменены с {old_languages} на {new_languages}")


class MarketingProject(Project, LoggingMixin, NotificationMixin):
    """Класс маркетингового проекта."""

    def __init__(self, name: str, startDate: datetime, endDate: datetime,
                 budget: float, status: str, team: 'Team', target_audience: int):
        """Инициализирует маркетинговый проект.

        Args:
            target_audience: Размер целевой аудитории
        """
        super().__init__(name, startDate, endDate, budget, status, team)
        self.target_audience = target_audience

    def to_dict(self) -> Dict[str, Any]:
        data = super().to_dict()
        data.update({
            'type': 'MarketingProject',
            'target_audience': self.target_audience
        })
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'MarketingProject':
        team = Team.from_dict(data['team'])  # Десериализация Team
        return cls(
            name=data['name'],
            startDate=datetime.fromisoformat(data['start_date']),
            endDate=datetime.fromisoformat(data['end_date']),
            budget=data['budget'],
            status=data['status'],
            team=team,
            target_audience=data['target_audience']
        )

    def calculate_progress(self) -> float:
        """Вычисляет прогресс с учетом размера целевой аудитории.

        Returns:
            Процент выполнения (0-100)
        """
        base_progress = super().calculate_progress()
        audience_factor = 1 - (min(self.target_audience, 1000000) / 10000000)
        return min(100, base_progress * audience_factor)

    def complete_project(self) -> None:
        """Отмечает проект как завершенный и отправляет уведомление."""
        self.status = "completed"
        self.send_notification("Проект успешно завершен!")


class ResearchProject(Project, LoggingMixin, NotificationMixin):
    """Класс исследовательского проекта."""

    def __init__(self, name: str, startDate: datetime, endDate: datetime,
                 budget: float, status: str, team: 'Team', research_field: str):
        """Инициализирует исследовательский проект.

        Args:
            research_field: Область исследования
        """
        super().__init__(name, startDate, endDate, budget, status, team)
        self.research_field = research_field

    def to_dict(self) -> Dict[str, Any]:
        data = super().to_dict()
        data.update({
            'type': 'ResearchProject',
            'research_field': self.research_field
        })
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'ResearchProject':
        team = Team.from_dict(data['team'])  # Десериализация Team
        return cls(
            name=data['name'],
            startDate=datetime.fromisoformat(data['start_date']),
            endDate=datetime.fromisoformat(data['end_date']),
            budget=data['budget'],
            status=data['status'],
            team=team,
            research_field=data['research_field']
        )

    def calculate_progress(self) -> float:
        """Вычисляет прогресс с учетом сложности области исследования.

        Returns:
            Процент выполнения (0-100)
        """
        base_progress = super().calculate_progress()
        complexity = {
            "математика": 0.9,
            "физика": 0.8,
            "биология": 0.7,
            "искусственный интеллект": 0.6,
            "квантовые вычисления": 0.5
        }.get(self.research_field.lower(), 0.7)
        return min(100, base_progress * complexity)

    def publish_paper(self, title: str) -> None:
        """Публикует научную статью и логирует событие.

        Args:
            title: Название статьи
        """
        self.log_action(f"Опубликована статья: '{title}'")
        self.send_notification(f"Новая публикация в области {self.research_field}!")


def save_projects(projects: List[Project], filename: str):
    """Сохраняет список проектов в JSON файл."""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump([p.to_dict() for p in projects], f, indent=2, ensure_ascii=False)

def load_projects(filename: str) -> List[Project]:
    """Загружает проекты из JSON файла."""
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    projects = []
    for item in data:
        if item['type'] == 'SoftwareProject':
            projects.append(SoftwareProject.from_dict(item))
        elif item['type'] == 'MarketingProject':
            projects.append(MarketingProject.from_dict(item))
        elif item['type'] == 'ResearchProject':
            projects.append(ResearchProject.from_dict(item))
        else:
            projects.append(Project.from_dict(item))
    return projects


class Employee:
    """Класс сотрудника."""
    def __init__(self, name: str, position: str, email: str):
        """Инициализирует сотрудника.
        Args:
            name: Имя сотрудника
            position: Должность
            email: Электронная почта
        """
        self.name = name
        self.position = position
        self.email = email

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует сотрудника в словарь.
        Returns:
            Словарь с данными сотрудника
        """
        return {
            'name': self.name,
            'position': self.position,
            'email': self.email
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]):
        """Создает Employee из словаря.
        Args:
            data: Словарь с данными сотрудника
        Returns:
            Экземпляр Employee
        """
        return cls(
            name=data['name'],
            position=data['position'],
            email=data['email']
        )

class Team:
    """Класс команды проекта."""
    def __init__(self, name: str, leader: Employee):
        """Инициализирует команду.
        Args:
            name: Название команды
            leader: Руководитель команды
        """
        self.name = name
        self.leader = leader
        self.members = []

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует команду в словарь.
        Returns:
            Словарь с данными команды
        """
        return {
            'name': self.name,
            'leader': self.leader.to_dict(),
            'members': [member.to_dict() for member in self.members]
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]):
        """Создает Team из словаря.
        Args:
            data: Словарь с данными команды
        Returns:
            Экземпляр Team
        """
        leader = Employee.from_dict(data['leader'])
        team = cls(name=data['name'], leader=leader)
        team.members = [Employee.from_dict(member_data) for member_data in data['members']]
        return team

    def add_member(self, member: Employee):
        """Добавляет участника в команду.

        Args:
            member: Участник для добавления
        """
        if member not in self.members:
            self.members.append(member)
        else:
            print(f"Участник {member.name} уже в команде")

    def remove_member(self, member: Employee):
        """Удаляет участника из команды.

        Args:
            member: Участник для удаления
        """
        if member in self.members:
            self.members.remove(member)
        else:
            print(f"Участник {member.name} не найден в команде")

    def get_members(self):
        """Возвращает список всех участников команды (включая руководителя).

        Returns:
            Список участников
        """
        return [self.leader] + self.members

    def get_team_size(self):
        """Возвращает общее количество участников команды.

        Returns:
            Количество участников
        """
        return len(self.members) + 1

    def __str__(self) -> str:
        """Строковое представление команды.

        Returns:
            Форматированная строка с информацией о команде
        """
        members_list = "\n".join([f"{member.name}" for member in self.get_members()])
        return f"Команда: {self.name}\nУчастники:\n{members_list}\nВсего участников: {self.get_team_size()}"


def main():
    """Основная функция для демонстрации работы системы управления проектами."""
    import logging
    from datetime import datetime, timedelta

    # Настройка логирования
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler("project_management.log"),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    try:
        # Создаем сотрудников
        emp1 = Employee("Иван Петров", "Разработчик", "ivan@example.com")
        emp2 = Employee("Мария Сидорова", "Маркетолог", "maria@example.com")
        emp3 = Employee("Алексей Иванов", "Исследователь", "alex@example.com")
        emp4 = Employee("Елена Смирнова", "Руководитель", "elena@example.com")

        # Создаем команды
        dev_team = Team("Разработка", emp4)
        dev_team.add_member(emp1)

        marketing_team = Team("Маркетинг", emp4)
        marketing_team.add_member(emp2)

        research_team = Team("Исследования", emp4)
        research_team.add_member(emp3)

        # Используем фабрику проектов
        project_factory = ProjectFactory()

        # Создаем проекты разных типов
        today = datetime.now()
        software_project = project_factory.create_project(
            "software",
            "Разработка CRM",
            today,
            today + timedelta(days=30),
            15000.0,
            "active",
            dev_team,
            ["Python", "JavaScript"]
        )

        marketing_project = project_factory.create_project(
            "marketing",
            "Рекламная кампания",
            today,
            today + timedelta(days=60),
            8000.0,
            "planning",
            marketing_team,
            500000
        )

        research_project = project_factory.create_project(
            "research",
            "Исследование ИИ",
            today,
            today + timedelta(days=90),
            20000.0,
            "active",
            research_team,
            "искусственный интеллект"
        )

        # Демонстрация работы проектов
        print("\n=== Информация о проектах ===")
        print(software_project)
        print(marketing_project)
        print(research_project)

        # Отслеживание прогресса
        print("\n=== Прогресс проектов ===")
        print(f"{software_project.name}: {software_project.track_progress()}")
        print(f"{marketing_project.name}: {marketing_project.track_progress()}")
        print(f"{research_project.name}: {research_project.track_progress()}")

        # Работа с командой
        print("\n=== Управление командой ===")
        print(f"Команда разработки до добавления: {dev_team.get_team_size()} участников")
        new_dev = Employee("Сергей Козлов", "Разработчик", "sergey@example.com")
        dev_team.add_member(new_dev)
        print(f"Команда разработки после добавления: {dev_team.get_team_size()} участников")
        print(dev_team)

        # Логирование и уведомления
        print("\n=== Логирование и уведомления ===")
        software_project.update_languages(["Python", "JavaScript", "TypeScript"])
        marketing_project.complete_project()
        research_project.publish_paper("Новые алгоритмы ИИ")

        # Цепочка ответственности для бюджета
        print("\n=== Обработка запросов на бюджет ===")
        project_leader = ProjectLeader()
        finance_dept = FinanceDepartment()
        director = Director()

        # Настраиваем цепочку
        project_leader.set_next(finance_dept).set_next(director)

        # Тестируем запросы
        requests = [
            {"amount": 800},   # Руководитель проекта
            {"amount": 2500},  # Финансовый отдел
            {"amount": 7500},  # Директор
            {"amount": 15000}  # Никто не обработает
        ]

        for request in requests:
            print(f"\nОбработка запроса на {request['amount']}:")
            project_leader.decision_request(request)

        # Сохранение и загрузка проектов
        print("\n=== Сохранение и загрузка проектов ===")
        projects = [software_project, marketing_project, research_project]
        save_projects(projects, "projects.json")
        loaded_projects = load_projects("projects.json")
        print(f"Загружено {len(loaded_projects)} проектов")

        # Проверка загруженных данных
        for project in loaded_projects:
            print(f"Загружен проект: {project.name} ({project.status})")

        # Обработка исключений
        print("\n=== Обработка исключений ===")
        try:
            # Некорректные даты проекта
            validate_project_dates(
                today + timedelta(days=10),
                today
            )
        except InvalidDateError as e:
            logger.error(f"Ошибка даты: {e}")

        try:
            # Недостаточно прав
            user = {"name": "Тестовый пользователь", "permissions": []}

            @check_permissions("edit_project")
            def edit_project(user):
                print("Проект отредактирован")

            edit_project(user)
        except PermissionDeniedError as e:
            logger.error(f"Ошибка прав: {e}")

    except Exception as e:
        logger.error(f"Произошла ошибка: {e}", exc_info=True)

if __name__ == "__main__":
    main()

ERROR:__main__:Ошибка даты: Дата окончания (2025-04-24 11:03:34.941157) не может быть раньше даты начала (2025-05-04 11:03:34.941157)
ERROR:__main__:Ошибка прав: У пользователя Тестовый пользователь нет права edit_project



=== Информация о проектах ===
Название проекта: Разработка CRM. Статус: active.
Название проекта: Рекламная кампания. Статус: planning.
Название проекта: Исследование ИИ. Статус: active.

=== Прогресс проектов ===
Разработка CRM: Прогресс: 45.0% завершено
Рекламная кампания: Прогресс: 47.5% завершено
Исследование ИИ: Прогресс: 30.0% завершено

=== Управление командой ===
Команда разработки до добавления: 2 участников
Команда разработки после добавления: 3 участников
Команда: Разработка
Участники:
Елена Смирнова
Иван Петров
Сергей Козлов
Всего участников: 3

=== Логирование и уведомления ===
🔔 [Лог] Проект 'Разработка CRM' обновлен: Языки изменены с ['Python', 'JavaScript'] на ['Python', 'JavaScript', 'TypeScript']
🔔 [Уведомление] Проект 'Рекламная кампания': Проект успешно завершен!
🔔 [Лог] Проект 'Исследование ИИ' обновлен: Опубликована статья: 'Новые алгоритмы ИИ'
🔔 [Уведомление] Проект 'Исследование ИИ': Новая публикация в области искусственный интеллект!

=== Обработка запросов на